# Análisis Exploratorio de Datos - Matrimonios en Guatemala
## CC3074 - Minería de Datos
## Universidad del Valle de Guatemala
### Semestre I - 2026

---

## 1. Descripción General del Conjunto de Datos

### 1.1 Introducción

Este análisis se basa en los datos de **matrimonios civiles** registrados en Guatemala, obtenidos del Instituto Nacional de Estadística (INE). El conjunto de datos consolidado abarca **múltiples años** (más de 10 años de registros) y contiene información detallada sobre las características demográficas, educativas y geográficas de los matrimonios celebrados en el país.

**Fuente de datos:** Instituto Nacional de Estadística (INE) - Estadísticas Vitales  
**Formato original:** Archivos .sav (SPSS) por año  
**Periodo:** Multi-año (2009-2022 aproximadamente)  
**Proceso de consolidación:** Se unificaron múltiples años tomando únicamente las variables comunes en todos los periodos para garantizar consistencia temporal.

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [4]:
# Cargar los datos
# NOTA: Ajusta la ruta según donde tengas tu archivo
df = pd.read_csv('mat_unido_columnas_comunes.csv')

print("✓ Datos cargados exitosamente")
print(f"Forma del dataset: {df.shape}")

✓ Datos cargados exitosamente
Forma del dataset: (752264, 15)


### 1.2 Dimensiones del Dataset

In [6]:
# Información general del dataset
print(f"\nNúmero de observaciones (registros): {df.shape[0]:,}")
print(f"Número de variables (columnas): {df.shape[1]}")
print(f"Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nPeriodo de análisis: {int(df['AÑOREG'].min())} - {int(df['AÑOREG'].max())}")
print(f"Años de datos: {int(df['AÑOREG'].max() - df['AÑOREG'].min() + 1)} años")


Número de observaciones (registros): 752,264
Número de variables (columnas): 15
Tamaño en memoria: 108.29 MB

Periodo de análisis: 2011 - 2021
Años de datos: 11 años


In [8]:
# Crear diccionario detallado de variables
diccionario_variables = {
    'Variable': [
        'AÑOREG', 'MESREG', 'MESOCU', 'DIAOCU',  # Temporales
        'DEPREG', 'MUPREG', 'DEPOCU', 'MUPOCU',  # Geográficas
        'CLAUNI',  # Matrimonio
        'EDADHOM', 'ESCHOM', 'NACHOM',  # Hombre
        'EDADMUJ', 'ESCMUJ', 'NACMUJ'   # Mujer
    ],
    'Nombre_Completo': [
        'Año de Registro', 'Mes de Registro', 'Mes de Ocurrencia', 'Día de Ocurrencia',
        'Departamento de Registro', 'Municipio de Registro', 'Departamento de Ocurrencia', 'Municipio de Ocurrencia',
        'Clase de Unión',
        'Edad del Hombre', 'Escolaridad del Hombre', 'Nacionalidad del Hombre',
        'Edad de la Mujer', 'Escolaridad de la Mujer', 'Nacionalidad de la Mujer'
    ],
    'Tipo': [
        'Numérica Discreta', 'Categórica Ordinal', 'Categórica Ordinal', 'Numérica Discreta',
        'Categórica Nominal', 'Categórica Nominal', 'Categórica Nominal', 'Categórica Nominal',
        'Categórica Nominal',
        'Numérica Continua', 'Categórica Ordinal', 'Categórica Nominal',
        'Numérica Continua', 'Categórica Ordinal', 'Categórica Nominal'
    ],
    'Categoría': [
        'Temporal', 'Temporal', 'Temporal', 'Temporal',
        'Geográfica', 'Geográfica', 'Geográfica', 'Geográfica',
        'Matrimonio',
        'Hombre', 'Hombre', 'Hombre',
        'Mujer', 'Mujer', 'Mujer'
    ],
    'Descripción': [
        'Año en que se inscribió el matrimonio en el registro civil',
        'Mes en que se inscribió el matrimonio (1-12)',
        'Mes en que se celebró la ceremonia matrimonial (1-12)',
        'Día del mes en que se celebró el matrimonio (1-31)',
        'Código del departamento donde se registró (1-22)',
        'Código del municipio donde se registró (101-2217)',
        'Código del departamento donde se celebró (1-22)',
        'Código del municipio donde se celebró (101-2217)',
        'Tipo de matrimonio: 1=Civil, 2=Religioso, 3=Mixto, 9=No especificado',
        'Edad en años del contrayente masculino',
        'Nivel educativo: 1=Ninguno, 2=Primaria, 3=Básicos, 4=Diversificado, 5=Universidad, 6=Posgrado, 9=No especificado',
        'Código ISO del país de nacionalidad (320=Guatemala)',
        'Edad en años de la contrayente femenina',
        'Nivel educativo: 1=Ninguno, 2=Primaria, 3=Básicos, 4=Diversificado, 5=Universidad, 6=Posgrado, 9=No especificado',
        'Código ISO del país de nacionalidad (320=Guatemala)'
    ]
}

df_diccionario = pd.DataFrame(diccionario_variables)
display(df_diccionario)

,Variable,Nombre_Completo,Tipo,Categoría,Descripción
0,AÑOREG,Año de Registro,Numérica Discreta,Temporal,Año en que se inscribió el matrimonio en el re...
1,MESREG,Mes de Registro,Categórica Ordinal,Temporal,Mes en que se inscribió el matrimonio (1-12)
2,MESOCU,Mes de Ocurrencia,Categórica Ordinal,Temporal,Mes en que se celebró la ceremonia matrimonial...
3,DIAOCU,Día de Ocurrencia,Numérica Discreta,Temporal,Día del mes en que se celebró el matrimonio (1...
4,DEPREG,Departamento de Registro,Categórica Nominal,Geográfica,Código del departamento donde se registró (1-22)
5,MUPREG,Municipio de Registro,Categórica Nominal,Geográfica,Código del municipio donde se registró (101-2217)
6,DEPOCU,Departamento de Ocurrencia,Categórica Nominal,Geográfica,Código del departamento donde se celebró (1-22)
7,MUPOCU,Municipio de Ocurrencia,Categórica Nominal,Geográfica,Código del municipio donde se celebró (101-2217)
8,CLAUNI,Clase de Unión,Categórica Nominal,Matrimonio,"Tipo de matrimonio: 1=Civil, 2=Religioso, 3=Mi..."
9,EDADHOM,Edad del Hombre,Numérica Continua,Hombre,Edad en años del contrayente masculino


### 1.4 Clasificación de Variables por Tipo

In [17]:
# Clasificar variables por tipo
variables_numericas = ['AÑOREG', 'DIAOCU', 'EDADHOM', 'EDADMUJ', 'MUPREG', 'MUPOCU']
variables_categoricas_ordinales = ['MESREG', 'MESOCU', 'ESCHOM', 'ESCMUJ']
variables_categoricas_nominales = ['DEPREG', 'DEPOCU', 'CLAUNI', 'NACHOM', 'NACMUJ']

print("Clasificacion de variables por tipo")

print(f"\nVariables Numéricas ({len(variables_numericas)}):")
for var in variables_numericas:
    nombre = df_diccionario[df_diccionario['Variable']==var]['Nombre_Completo'].values[0]
    print(f"   • {var}: {nombre}")

print(f"\nVariables Categóricas Ordinales ({len(variables_categoricas_ordinales)}):")
for var in variables_categoricas_ordinales:
    nombre = df_diccionario[df_diccionario['Variable']==var]['Nombre_Completo'].values[0]
    print(f"   • {var}: {nombre}")

print(f"\nVariables Categóricas Nominales ({len(variables_categoricas_nominales)}):")
for var in variables_categoricas_nominales:
    nombre = df_diccionario[df_diccionario['Variable']==var]['Nombre_Completo'].values[0]
    print(f"   • {var}: {nombre}")

print(f"\nTotal de variables: {len(variables_numericas) + len(variables_categoricas_ordinales) + len(variables_categoricas_nominales)}")

Clasificacion de variables por tipo

Variables Numéricas (6):
   • AÑOREG: Año de Registro
   • DIAOCU: Día de Ocurrencia
   • EDADHOM: Edad del Hombre
   • EDADMUJ: Edad de la Mujer
   • MUPREG: Municipio de Registro
   • MUPOCU: Municipio de Ocurrencia

Variables Categóricas Ordinales (4):
   • MESREG: Mes de Registro
   • MESOCU: Mes de Ocurrencia
   • ESCHOM: Escolaridad del Hombre
   • ESCMUJ: Escolaridad de la Mujer

Variables Categóricas Nominales (5):
   • DEPREG: Departamento de Registro
   • DEPOCU: Departamento de Ocurrencia
   • CLAUNI: Clase de Unión
   • NACHOM: Nacionalidad del Hombre
   • NACMUJ: Nacionalidad de la Mujer

Total de variables: 15


### 1.5 Vista Previa de los Datos

In [10]:
display(df.head(10))

,AÑOREG,CLAUNI,DEPOCU,DEPREG,DIAOCU,EDADHOM,EDADMUJ,ESCHOM,ESCMUJ,MESOCU,MESREG,MUPOCU,MUPREG,NACHOM,NACMUJ
0,2011.0,3.0,1.0,1.0,20.0,27.0,32.0,9.0,5.0,9.0,12.0,101,101,124.0,320.0
1,2012.0,2.0,1.0,1.0,12.0,29.0,31.0,1.0,1.0,11.0,1.0,108,108,320.0,320.0
2,2011.0,3.0,8.0,8.0,12.0,21.0,17.0,9.0,1.0,12.0,12.0,801,801,320.0,320.0
3,2011.0,2.0,1.0,1.0,25.0,58.0,53.0,1.0,1.0,10.0,11.0,101,101,320.0,320.0
4,2011.0,3.0,1.0,1.0,2.0,31.0,24.0,5.0,4.0,10.0,11.0,103,103,320.0,320.0
5,2012.0,3.0,1.0,6.0,16.0,27.0,25.0,4.0,4.0,12.0,1.0,109,608,320.0,320.0
6,2011.0,9.0,5.0,5.0,9.0,27.0,24.0,5.0,4.0,7.0,7.0,507,507,320.0,320.0
7,2011.0,3.0,1.0,1.0,11.0,29.0,50.0,5.0,3.0,3.0,3.0,102,102,340.0,320.0
8,2011.0,3.0,1.0,1.0,16.0,38.0,30.0,5.0,5.0,4.0,7.0,102,102,40.0,320.0
9,2011.0,9.0,10.0,10.0,14.0,32.0,23.0,1.0,4.0,9.0,10.0,1001,1001,320.0,320.0


In [12]:
print(df.dtypes)

AÑOREG     float64
CLAUNI     float64
DEPOCU     float64
DEPREG     float64
DIAOCU     float64
EDADHOM    float64
EDADMUJ    float64
ESCHOM     float64
ESCMUJ     float64
MESOCU     float64
MESREG     float64
MUPOCU      object
MUPREG       int64
NACHOM     float64
NACMUJ     float64
dtype: object


### 1.7 Estadísticas Descriptivas Básicas

In [15]:
# Resumen estadístico de variables numéricas
display(df[variables_numericas].describe())

,AÑOREG,DIAOCU,EDADHOM,EDADMUJ,MUPREG
count,752264.000000,752264.000000,752264.000000,752264.000000,752264.000000
mean,2015.404325,16.034647,29.744796,26.796582,958.986192
std,2.844226,8.645327,22.286581,22.764776,640.620477
min,2011.000000,1.000000,12.000000,10.000000,101.000000
25%,2013.000000,9.000000,22.000000,20.000000,401.000000
50%,2015.000000,16.000000,26.000000,23.000000,1002.000000
75%,2018.000000,23.000000,32.000000,29.000000,1415.000000
max,2021.000000,31.000000,999.000000,999.000000,2217.000000


In [25]:
# Frecuencias de variables categóricas clave
print("DISTRIBUCIÓN DE VARIABLES CATEGÓRICAS CLAVE")

# Clase de Unión
print("\nCLASE DE UNIÓN (CLAUNI)")
clauni_counts = df['CLAUNI'].value_counts().sort_index()
clauni_pct = (df['CLAUNI'].value_counts(normalize=True).sort_index() * 100).round(2)

# Mapeo correcto para CLAUNI
clauni_map = {1: 'Civil', 2: 'Religioso', 3: 'Mixto', 9: 'No especificado'}
clauni_df = pd.DataFrame({
    'Código': clauni_counts.index,
    'Tipo': [clauni_map.get(int(i), 'Desconocido') for i in clauni_counts.index],
    'Frecuencia': clauni_counts.values,
    'Porcentaje': clauni_pct.values
})
display(clauni_df)

# Escolaridad Hombre
print("\nESCOLARIDAD DEL HOMBRE (ESCHOM)")
eschom_counts = df['ESCHOM'].value_counts().sort_index()
eschom_pct = (df['ESCHOM'].value_counts(normalize=True).sort_index() * 100).round(2)

# Mapeo correcto para escolaridad
niveles_educativos_map = {
    1: 'Ninguno',
    2: 'Primaria',
    3: 'Básicos',
    4: 'Diversificado',
    5: 'Universidad',
    6: 'Posgrado',
    9: 'No especificado'
}

eschom_df = pd.DataFrame({
    'Código': eschom_counts.index,
    'Nivel': [niveles_educativos_map.get(int(i), 'Desconocido') for i in eschom_counts.index],
    'Frecuencia': eschom_counts.values,
    'Porcentaje': eschom_pct.values
})
display(eschom_df)

# Escolaridad Mujer
print("\n   ESCOLARIDAD DE LA MUJER (ESCMUJ)")
escmuj_counts = df['ESCMUJ'].value_counts().sort_index()
escmuj_pct = (df['ESCMUJ'].value_counts(normalize=True).sort_index() * 100).round(2)

escmuj_df = pd.DataFrame({
    'Código': escmuj_counts.index,
    'Nivel': [niveles_educativos_map.get(int(i), 'Desconocido') for i in escmuj_counts.index],
    'Frecuencia': escmuj_counts.values,
    'Porcentaje': escmuj_pct.values
})
display(escmuj_df)

DISTRIBUCIÓN DE VARIABLES CATEGÓRICAS CLAVE

CLASE DE UNIÓN (CLAUNI)


,Código,Tipo,Frecuencia,Porcentaje
0,1.0,Civil,42186,5.61
1,2.0,Religioso,15381,2.04
2,3.0,Mixto,666351,88.58
3,9.0,No especificado,28346,3.77



ESCOLARIDAD DEL HOMBRE (ESCHOM)


,Código,Nivel,Frecuencia,Porcentaje
0,1.0,Ninguno,103627,13.78
1,2.0,Primaria,256873,34.15
2,3.0,Básicos,127614,16.96
3,4.0,Diversificado,198531,26.39
4,5.0,Universidad,29578,3.93
5,6.0,Posgrado,212,0.03
6,9.0,No especificado,35829,4.76



   ESCOLARIDAD DE LA MUJER (ESCMUJ)


,Código,Nivel,Frecuencia,Porcentaje
0,1.0,Ninguno,152476,20.27
1,2.0,Primaria,225986,30.04
2,3.0,Básicos,104866,13.94
3,4.0,Diversificado,192845,25.64
4,5.0,Universidad,25380,3.37
5,6.0,Posgrado,140,0.02
6,9.0,No especificado,50571,6.72
